In [3]:
import requests
from bs4 import BeautifulSoup
resp = requests.get('http://blog.castman.net/web-crawler-tutorial/ch2/table/table.html')
soup = BeautifulSoup(resp.text,'html.parser')

#計算課程平均價格
#取得課程所有價格,使用index
#方法一

prices=[]
rows = soup.find('table','table').tbody.find_all('tr')
for row in rows:
    price = row.find_all('td')[2].text
    prices.append(int(price))

print('均價: ',sum(prices)/len(prices))

均價:  1823.3333333333333


In [5]:
#方法二

prices=[]
links=soup.find_all('a')
for link in links:
    price = link.parent.previous_sibling.text
    prices.append(int(price))
    
print('均價: ',sum(prices)/len(prices))

均價:  1823.3333333333333


In [12]:
#取得每一列所有欄位資訊

rows = soup.find('table','table').tbody.find_all('tr')
for row in rows:
    print([s for s in row.stripped_strings])








['初心者 - Python入門', '初學者', '1490']
['Python 網頁爬蟲入門實戰', '有程式基礎的初學者', '1890']
['Python 機器學習入門實戰 (預計)', '有程式基礎的初學者', '1890']
['Python 資料科學入門實戰 (預計)', '有程式基礎的初學者', '1890']
['Python 資料視覺化入門實戰 (預計)', '有程式基礎的初學者', '1890']
['Python 網站架設入門實戰 (預計)', '有程式基礎的初學者', '1890']


In [14]:
rows = soup.find('table','table').tbody.find_all('tr')
for row in rows:
    all_tds = row.find_all('td')
    if 'href' in all_tds[3].a.attrs:
        href = all_tds[3].a['href']
    else:
        herf = None
    print(all_tds[0].text,all_tds[1].text,all_tds[2].text,href,all_tds[3].a.img['src'])

初心者 - Python入門 初學者 1490 http://www.pycone.com img/python-logo.png
Python 網頁爬蟲入門實戰 有程式基礎的初學者 1890 http://www.pycone.com img/python-logo.png
Python 機器學習入門實戰 (預計) 有程式基礎的初學者 1890 http://www.pycone.com img/python-logo.png
Python 資料科學入門實戰 (預計) 有程式基礎的初學者 1890 http://www.pycone.com img/python-logo.png
Python 資料視覺化入門實戰 (預計) 有程式基礎的初學者 1890 http://www.pycone.com img/python-logo.png
Python 網站架設入門實戰 (預計) 有程式基礎的初學者 1890 http://www.pycone.com img/python-logo.png


In [15]:
html='<div data-name="123">foo<div>'
soup=BeautifulSoup(html,'html.parser')
data_tag = soup.find(attrs={'data-name','123'})
print(data_tag)

None


In [33]:
import requests
import re
from bs4 import BeautifulSoup
Y_MOVIE_URL = 'https://movies.yahoo.com.tw/movie_thisweek.html'
def get_web_page(url):
    resp = requests.get(url)
    if resp.status_code != 200:
        print('Invaid url',resp.url)
        return None
    else:
        return resp.text


def get_data(data_str):
    pattern = '\d+-\d+-\d+'
    match = re.search(pattern,data_str)
    if match != None:
        return data_str
    else:
        return match.group(0)
    
def get_movie_id(url):
    try:
        movie_id = url.split('.html')[0].split('-')[-1]
    except:
        movie_id = url
    return movie_id
    
# if __name__ == '__main__':
#     page = get_web_page(Y_MOVIE_URL)
#     print(get_movie_id('https://movies.yahoo.com.tw/movieinfo_main/%E8%A8%98%E6%86%B6%E5%B1%8B-kiokuya-10470'))

In [35]:
def get_movies(dom):
    soup = BeautifulSoup(dom,'html.parser')
    movies=[]
    rows = soup.find_all('div','release_info_text')
    for row in rows:
        movie = dict()
        movie['expection']=row.find('div','leveltext').span.text.strip()
        movie['ch_name']=row.find('div','release_movie_name').a.text.strip()
        movie['eng_name']=row.find('div','release_movie_name').find('div','en').a.text.strip()
        movie['movie_id']=get_movie_id(row.find('div','release_movie_name').a['href'])
        movie['poster_url']=row.parent.find_previous_sibling('div','release_foto').a.img['src']
        movie['release_data'] = get_data(row.find('div','release_movie_time').text)
        movies.append(movie)
    return movies

if __name__ == '__main__':
    page = get_web_page(Y_MOVIE_URL)
    movies = get_movies(page)
    for movie in movies:
        print(movie)

{'expection': '87%', 'ch_name': '記憶屋', 'eng_name': 'kiokuya', 'movie_id': '10470', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/January2020/V6MY1SapAqTFmjnS4ChM-800x1138.jpg', 'release_data': '上映日期 ： 2020-03-27'}
{'expection': '89%', 'ch_name': 'Re從零開始的異世界生活 外傳集', 'eng_name': '', 'movie_id': '10480', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/January2020/TDRN5rFEzoyTnUEnQ6EN-500x720.jpg', 'release_data': '上映日期 ： 2020-03-27'}
{'expection': '88%', 'ch_name': '第六感生死戀', 'eng_name': 'Ghost', 'movie_id': '10504', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/March2020/ygqvTlkSMVgefHs3LplV-1000x1424.jpg', 'release_data': '上映日期 ： 2020-03-27'}
{'expection': '82%', 'ch_name': '抓住救命稻草的野獸們', 'eng_name': 'Beasts Clawing at Straws', 'movie_id': '10507', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/February2020/uxiNS7oVZZ92gPsGXJl5-1280x1834.jpg', 'release_data': '上映日期 ： 2020-03-27'}
{

expection


In [46]:
import requests
from bs4 import BeautifulSoup

def append_list_pm25():
    url = 'https://taqm.epa.gov.tw/pm25/tw/PM25A.aspx?area=1'
    html = requests.get(url)
    sp = BeautifulSoup(html.text, 'html.parser')
    rs = sp.find_all("tr", {"align": "center", "style": "border-width:1px;border-style:Solid;"})
    for r in rs:
        name = r.find('a')
        pm25 = r.find_all('span')
        dic = {}
        dic.setdefault('name',   name.text.strip())
        dic.setdefault('pm25',   pm25[0].text.strip())
        dic.setdefault('pm25_1', pm25[1].text.strip())
        list.append(dic)

def get_pm25(name):
    for d in list:
        if d.get('name') == name:
            return d
list = []
append_list_pm25()

name = input("請輸入地區 : ")
d = get_pm25(name)
print(d)
print(d.get("pm25"))

請輸入地區 : 桃園
{'name': '桃園', 'pm25': '37', 'pm25_1': '33'}
37


In [50]:
import requests
from bs4 import BeautifulSoup
G_FINANCE_URL = 'https://www.google.com/search?q=2330'
def get_web_page(url,stock_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                             'AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/66.0.3359.181 Safari/537.36'}  #請求頭 包含大多數的網頁瀏覽器
    resp = requests.get(url + stock_id,headers=headers)
    if resp.status_code != 200:
        print("找不到網頁:",resp.url)
        return None
    else:
        return resp.text